In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Set hyperparameters for training
batch_size = 64  # Number of sequences processed in parallel
block_size = 256  # Maximum context length for predictions
max_iters = 5  # Total number of iterations for training
eval_interval = 500  # Interval at which to evaluate the model
learning_rate = 3e-4  # Learning rate for the optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
eval_iters = 200  # Number of iterations for evaluation
n_embd = 384  # Embedding dimension
n_head = 6  # Number of heads in multi-head attention
n_layer = 6  # Number of Transformer blocks
dropout = 0.2  # Dropout rate to prevent overfitting

# Set random seed for reproducibility
torch.manual_seed(1337)


### Data Preparation

In [46]:
# Load the input text file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Extract unique characters from the text
chars = sorted(list(set(text)))
vocab_size = len(chars)  # Number of unique characters

# Create mappings from characters to integers and vice versa
stoi = {ch: i for i, ch in enumerate(chars)}  # String to integer
itos = {i: ch for i, ch in enumerate(chars)}  # Integer to string

# Define encoding and decoding functions
encode = lambda s: [stoi[c] for c in s]  # Convert string to list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # Convert list of integers to string

# Encode the entire text as a list of integers
data = torch.tensor(encode(text), dtype=torch.long)

# Split data into training and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


### Data Loading Function

In [47]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    if len(data) <= block_size:
        raise ValueError("Data length is smaller than or equal to the block size. Increase the data length or reduce the block size.")
    
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


### Loss Estimation

In [48]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()  # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()  # Set model back to training mode
    return out


## Model Components

In [49]:
class Head(nn.Module):
    """ One head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B, T, head_size)
        q = self.query(x)  # (B, T, head_size)
        # Compute attention scores
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5  # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # Mask out future positions
        wei = F.softmax(wei, dim=-1)  # Normalize attention scores
        wei = self.dropout(wei)
        v = self.value(x)  # (B, T, head_size)
        out = wei @ v  # (B, T, head_size)
        return out


In [50]:
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # Concatenate outputs from all heads
        out = self.dropout(self.proj(out))  # Apply linear projection
        return out


In [51]:
class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)  # Apply feedforward network


In [52]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  # Add & Norm for self-attention
        x = x + self.ffwd(self.ln2(x))  # Add & Norm for feedforward
        return x


In [53]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # Token embeddings
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # Positional embeddings
        x = tok_emb + pos_emb  # Sum of token and positional embeddings
        x = self.blocks(x)  # Pass through Transformer blocks
        x = self.ln_f(x)  # Apply final layer normalization
        logits = self.lm_head(x)  # Compute logits

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # Reshape logits for loss computation
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)  # Compute cross-entropy loss

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]  # Crop to the last block_size tokens
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]  # Focus on the last time step
            probs = F.softmax(logits, dim=-1)  # Convert logits to probabilities
            idx_next = torch.multinomial(probs, num_samples=1)  # Sample from the distribution
            idx = torch.cat((idx, idx_next), dim=1)  # Append sampled index to the sequence
        return idx


## Training Loop

In [ ]:
# Initialize the model and move it to the appropriate device
model = GPTLanguageModel()
model.to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')  # Print number of parameters

# Create an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)  # Evaluate the model
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')  # Get a batch of training data
    logits, loss = model(xb, yb)  # Forward pass
    optimizer.zero_grad(set_to_none=True)  # Reset gradients
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

# Generate text from the trained model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))
